# Progetto Titanic gruppo 5
### Spironelli Riccardo - Tedi Kumaraku - Davide Nizzetto - Nicola Bonan - Matteo Penasa

### La sfida:
**Nella sfida proposta su Kaggle ci è stato chiesto  di costruire un modello di previsione che rispondesse alla domanda: "Quali persone avrebbero avuto più possibilità di salvarsi nell'incidente del Titanic", utilizzando i dati dei passeggeri proposti dal sito(Nome, Età, Sesso, Classe, ecc.).**

In [ ]:
#import necessari
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pandasql import sqldf
def pysqldf(q): return sqldf(q, globals())

### 2. Analisi dati
#### Inizialmente abbiamo addestrato la macchina utilizando il file train.csv

In [ ]:
#creazione del dataframe
titanic_data = pd.read_csv('DataSet/train.csv')

#### Import di Seaborn
##### Libreria che permette di creare grafici avanzati per la rappresentazione di dati.
##### In base alla colorazione della cella del grafico possiamo capire il livello di correlazione tra i due dati, prendiamo in considerazione sia i dati positivamente correlati sia i dati negativamente correlati.
##### Esempio: se prendiamo la riga dei sopravvissuti (Survived) e la colonna della classe sociale (Pclass) noteremo che più la classe sociale è bassa meno sono le possibilità di sopravvivenza, mentre se prendiamo sempre a riga dei sopravvissuti (Survived) e la colonna della tariffa pagata (Fare) vedremo che più alta è la tariffa che ogni persona ha pagato più sono le sue possibilità di sopravvivenza, se prendiamo la riga dei sopravvissuti (Survived) e la colonna con il numero del biglietto (PassengerId) osserveremo che la loro correlazione è pressoché nulla

In [ ]:
import seaborn as sns

sns.heatmap(titanic_data.corr(),cmap="YlGnBu")

#### Siamo poi andati a mostrare graficamente la suddivisione del passeggeri per classe: 

In [ ]:
d1 = pysqldf("select Pclass, count(Pclass) as notSurvived from titanic_data where Survived = 0 group by Pclass")

plt.bar(["Prima classe", "Seconda classe", "Terza classe"], d1["notSurvived"])
plt.title("Numero di morti divisi per classe")

#### Rappresentazione grafica di sopravvissuti e morti per sesso 

In [ ]:
plot = sns.countplot(x="Survived",hue="Sex", data=titanic_data)

In [ ]:
g = sns.FacetGrid(titanic_data, col='Survived')
g.map(plt.hist, 'Age', bins=20,color="orange")

### 3) TestData
#### Creiamo due dataframe partendo dal file train.csv: le righe del dataframe iniziale vengono mescolate randomicamente poi l'80% dei dati viene memorizzato in dataframe (strat_train_set) per l'addestramento, e il restante 20% nel dataframe (strat_test_set) per testare le predizioni dell'algoritmo.

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split  = StratifiedShuffleSplit(n_splits=1,test_size=0.2)
for train_indices, test_indices in split.split(titanic_data,titanic_data[["Survived","Pclass","Sex"]]):
    strat_train_set = titanic_data.loc[train_indices]
    strat_test_set = titanic_data.loc[test_indices]


#### Controlliamo che la distribuzione dei dati tra i due dataframe sia omogenea.

In [ ]:
plt.subplot(1,2,1)
strat_train_set['Survived'].hist()
strat_train_set['Pclass'].hist()

plt.subplot(1, 2, 2)
strat_test_set['Survived'].hist()
strat_test_set['Pclass'].hist()


### 4) Parsing dati 
#### Analisi necessaria vista la mancanza di dati su colonne con dati rilevanti (la colonna age presenta 135 celle vuote)

In [ ]:
strat_train_set[["Survived","Age"]].info()

#### Creiamo le classi necessarie alla pipeline

#### AgeImputer va ad popolare i campi vuoti di age con valori neutri (media di age).

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer

class AgeImputer (BaseEstimator, TransformerMixin):
    
    def fit(self,X, y=None):
        return self
    
    def transform(self,X):
        imputer = SimpleImputer(strategy="mean")
        X["Age"] = imputer.fit_transform(X[["Age"]])
        return X
    

#### FeatureEncoder va a creare diverse colonne per suddividere i dati presenti nei campi Embarked e Sex.
#### Prima: 
![One-Hot Img](./Img/Prima_del_OneHot.png)
#### Dopo:
![One-Hot Img](./Img/OneHot.png)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

class FeatureEncoder (BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        encoder = OneHotEncoder()
        matrix = encoder.fit_transform(X[["Embarked"]]).toarray()
        
        column_names = ["C","S","Q","N"]
        
        for i in range(len(matrix.T)):
            X[column_names[i]] = matrix.T[i]
            
        matrix = encoder.fit_transform(X[["Sex"]]).toarray()
        column_names = ["Female","Male"]

        for i in range(len(matrix.T)):
            X[column_names[i]] = matrix.T[i]
            
            
        return X


#### FeatureDropper va a rimuovere le colonne originali utilizzate dal FeatureEncoder (Embarked e Sex) e inoltre elimina le colonne che riteniamo poco utili per la previsione

In [ ]:
class FeatureDropper (BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self,X):
        return X.drop(["Embarked","Name","Ticket","Cabin","Sex","N"],axis=1,errors="ignore")


### Addestramento train set
#### Creazione e Utilizzo della pipeline

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([("AgeImputer", AgeImputer()), ("FeatureEncoder",
                    FeatureEncoder()), ("FeatureDropper", FeatureDropper())])

strat_train_set = pipeline.fit_transform(strat_train_set)



In [ ]:
strat_train_set.head()


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X = strat_train_set.drop(["Survived"],axis=1)
Y = strat_train_set["Survived"]

X_data = scaler.fit_transform(X)
Y_data = Y.to_numpy()

#### 5) Diamo in pasto i dati all'algoritmo che abbiamo scelto: 
#### Random Forest:
![One-Hot Img](./Img/RandomForest.png)
#### Perché lo abbiamo scelto: 
* #### Facile da usare in confronto alle reti neurali
* #### Efficente
* #### Preciso

#### Come funziona: Random Forest fa crescere più alberi decisionali che vengono uniti insieme per una previsione più accurata.

#### È facile confondersi tra un singolo albero decisionale e una foresta decisionale. Sembra che una foresta decisionale sia un insieme di singoli alberi decisionali, ed è... più o meno vero. È un gruppo di alberi a decisione singola, ma tutti gli alberi sono mescolati insieme in modo casuale invece di alberi separati che crescono individualmente.

#### Quando si utilizza un normale albero decisionale, si inserirà un set di dati di addestramento con caratteristiche ed etichette e formulerà un insieme di regole che utilizzerà per fare previsioni. Se hai inserito le stesse informazioni nell'algoritmo Random Forest, selezionerà casualmente osservazioni e caratteristiche per costruire diversi alberi decisionali e quindi fare la media dei risultati.

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier()

param_grid = [{"n_estimators":[10,100,200,500],"max_depth":[None,5,10],"min_samples_split":[2,3,4]}]

grid_search = GridSearchCV(clf,param_grid,cv=3,scoring="accuracy",return_train_score=True)

grid_search.fit(X_data,Y_data) #80% di train

GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid=[{'max_depth': [None, 5, 10],
                          'min_samples_split': [2, 3, 4],
                          'n_estimators': [10, 100, 200, 500]}],
             return_train_score=True, scoring='accuracy')

#### Selezioniamo così il miglior risultato.

In [17]:
final_clf = grid_search.best_estimator_

In [18]:
final_clf

RandomForestClassifier(max_depth=5, min_samples_split=4, n_estimators=500)

### 6) Addestramento train set con dataframe completo

#### Riapplichiamo la pipeline al train set parziale 20%

In [19]:
strat_test_set = pipeline.fit_transform(strat_test_set) # applico la pipeline al dataset di training

X_test = strat_test_set.drop(["Survived"], axis=1)      # /
Y_test = strat_test_set["Survived"]                     # isoliamo la colonna "survived" dal train set

scaler = StandardScaler()
#utilizzato per standardizzare i dati
X_data_test = scaler.fit_transform(X_test)              # creo una matrice dal dataset normalizzando i dati
#fit -> per stimare la media del campione e la deviazione standard
#transform -> standardizzare i dati dell'insieme di training (X_test) usando i parametri appena calcolati dal metodo fit
Y_data_test = Y_test.to_numpy()                         # creo un array


#### Otteniamo così lo score dell'algoritmo

In [20]:
final_clf.score(X_data_test,Y_data_test)            # calcola il punteggio 

0.8044692737430168

#### Da qui in poi ricominciamo l'addestramento dell'algoritmo con il dataframe completo di "train"

In [21]:
final_data = pipeline.fit_transform(titanic_data)   # applico pipeline su dataset

In [22]:
X_final = final_data.drop(["Survived"], axis=1)     # /
Y_final = final_data["Survived"]                    # isoliamo la colonna "survived" dal dataset

scaler = StandardScaler()                       
X_data_final = scaler.fit_transform(X_final)        # creo una matrice dal dataset normalizzando i dati
Y_data_final = Y_final.to_numpy()                   # creo matrice/array.


In [23]:
prod_clf = RandomForestClassifier()                                                                     # crea oggetto

param_grid = [{"n_estimators":[10,100,200,500],"max_depth":[None,5,10],"min_samples_split":[2,3,4]}]    # parametri 
# estimators = n alberi da generare prima di restituire la predizione
# max_depth = n profondità dell'albero
# min_sample_split = numero di foglie necessario per dividere un nodo intero

grid_search = GridSearchCV(prod_clf,param_grid,cv=3,scoring="accuracy",return_train_score=True)         # gridsearch tecnica per cercare il miglior albero
# prod_clf => estimator object
# param_grid => parametri 
# cv => n split
# scoring => caratteristica da prediligere
# return train score => "true" : il punteggio di addestramento viene utilizzato per ottenere infomrazioni su come le diverse impostazioni dei parametri influiscono sul compromesso di overfitting/underfitting

grid_search.fit(X_data_final,Y_data_final)                                                              # uso algoritmo su dataset

GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid=[{'max_depth': [None, 5, 10],
                          'min_samples_split': [2, 3, 4],
                          'n_estimators': [10, 100, 200, 500]}],
             return_train_score=True, scoring='accuracy')

In [24]:
prod_final_clf = grid_search.best_estimator_     # viene selezionato l'estimator con più accuracy
prod_final_clf

RandomForestClassifier(max_depth=5, min_samples_split=3, n_estimators=200)

## Conclusione (test set)
#### Qui iniziamo la previsione del risultato con il file di test effettivo.

In [25]:
titanic_test_data = pd.read_csv("DataSet/test.csv")             #leggo il file test

final_test_data = pipeline.fit_transform(titanic_test_data)     #applico pipeline su dataset

In [26]:
X_final_test = final_test_data                              
X_final_test = X_final_test.fillna(method = "ffill")            # sostituisce i valori null

scaler = StandardScaler()                                  
X_data_final_test = scaler.fit_transform(X_final_test)          # creo una matrice dal dataset normalizzando i dati

predictions = prod_final_clf.predict(X_data_final_test)         # applico l'albero decisionale al dataset finale

#### Generiamo il file con i passeggeri sopravvissuti

In [27]:
final_df = pd.DataFrame(titanic_test_data["PassengerId"])       # memorizzo "PassengerId" nel datafreme 
final_df["Survived"] = predictions                              # aggiungo nella colonna "Survived" le predizioni
final_df.to_csv("DataSet/TitanicPredictions.csv", index=False)  # genero file csv da caricare su kaggle


In [28]:
final_df    

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
